In [1]:
# packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import scipy.stats as sps
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.distributions.empirical_distribution import ECDF
from hyppo.ksample import Energy, MMD, DISCO

import warnings
warnings.filterwarnings("ignore")

# Математическая статистика. Задача 1

Найдем точечную оценку с помощью второго момента методом моментов
\begin{align*}
    \mathbb{E}_{\lambda}\left(X_1^2\right) = \overline{X^2} &\Leftrightarrow \\
    (16\lambda^{*})^2 + 16\lambda^{*} - \overline{X^2} = 0 &\Leftrightarrow (\text{ т. к. параметр строго больше 0!})\\
    \lambda^{*} = \frac{\sqrt{4 * \overline{X^2} + 1} - 1}{32}.
\end{align*}
Осталось лишь использовать ее для оценки истинного параметра $\lambda$.

Код будет выглядеть следующим образом:

In [2]:
solution = lambda x: (np.sqrt(4 * (x ** 2).mean() + 1) - 1) / 32

А также давайте проведем валидацию и проверим mse:

In [3]:
def experiment(n=1000, m=100, mu=10) -> float:
    '''
    Здесь n - это размер каждой выборки, а m - это количество выборок.
    Мы для каждой выборки размера n считаем оценку, построенную методом моментов выше.
    В итоге получаем m оценок. Далее мы считаем mse оценок, 
    то есть среднееквадратичное отклонение от истинного параметра.
    '''

    samples = sps.poisson.rvs(mu=16 * mu, size=[m, n])
    estimates = np.array(list(map(solution, samples)))

    mse = ((mu - estimates) ** 2).mean()

    return mse

In [4]:
experiment()

0.0006724018319939851

# Математическая статистика. Задача 2

Итак, на вход нам мы ожидаем получить реализацию случайных величин:
\begin{align*}
    Z_i = \sqrt{X_i^2 + Y_i^2}, i=1,...,n,
\end{align*}
где $n$ - количество выпущенных Иваном стрел. Тогда заметим, что
\begin{align*}
    \sum\limits_{k=1}^{n} \frac{Z_k^2}{44\sigma^2} &= \sum\limits_{k=1}^{n} \left(\frac{X_i^2 + Y_i^2}{44\sigma^2}\right) \\
     &= \sum\limits_{k=1}^{n} \left(\left(\frac{X_i}{\sqrt{44\sigma^2}}\right)^2 + \left(\frac{Y_i}{\sqrt{44\sigma^2}}\right)^2\right) = \eta \sim {\cal{X}}_{2n}^2,
\end{align*}
т.е получили случайную величину, которая имеет распределение хи-квадрат с $2n$ степенями свободы (это распределение известное и уже не зависит от параметра!) Поэтому можно построить **точный** доверительный интервал уровня $1 - \alpha$ для неизвестного стандартного отклонения $\sigma$. Так как
\begin{align*}
    \mathbb{P}\left(x_{\alpha / 2, 2n}^2 < \eta < x_{1 - \alpha / 2, 2n}^2\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(x_{\alpha / 2, 2n}^2 < n \cdot \frac{\overline{Z^2}}{44\sigma^2} < x_{1 - \alpha / 2, 2n}^2\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2} < \sigma^2 < \frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(\sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2}} < \sigma < \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}}\right)  = 1 - \alpha,
\end{align*}
где $x_{1 - \alpha / 2, 2n}^2$ и $x_{\alpha / 2, 2n}^2$ квантили распределения хи-квадрат с $2n$ степенями свободы уровней $1 - \alpha / 2$ и $\alpha / 2$ соответственно, а также
\begin{align*}
    \overline{Z^2} = \frac{1}{n} \cdot \sum\limits_{k=1}^{n} Z_k^2.
\end{align*}
Поэтому **точный** доверительный интервал для $\sigma$ при заданном уровне доверия $1 - \alpha$ имеет вид:
\begin{align*}
    \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2}}, \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}}
\end{align*}

Давайте приведем корректное решение, а также сразу провалидируем его на симулированных данных:

In [5]:
def solution(p: float, x: np.array) -> tuple:
    '''
    Здесь мы просто будем возвращать доверительный интервал, который теоретически нашли выше.
    '''
    
    alpha = 1 - p
    size = len(x)
    left = np.sqrt(size * (x ** 2).mean() / (44 * sps.chi2.ppf(q=1 - alpha / 2, df=2 * size)))
    right = np.sqrt(size * (x ** 2).mean() / (44 * sps.chi2.ppf(q=alpha / 2, df=2 * size)))
    
    return left, \
           right


def experiment(n: int, std: float) -> pd.DataFrame:
    # вводим альфы и размеры выборок
    alphas = [0.99, 0.9, 0.7, 0.9, 0.95, 0.9]
    sample_sizes = [1000, 1000, 100, 100, 10, 10]

    # здесь будет список, в который будем складывать размер, доверие, частоту ошибок и длину интервала
    df = []
    
    data = zip(alphas, sample_sizes)

    for p, size in data:
        # создаем переменные для подсчета ошибок и для длин доверительных интервалов
        error_counter = 0
        ci_length = []
        
        for i in range(n):
            # симулируем выборки, состоящие из size расстояний
            sample = np.sqrt(sps.norm.rvs(0, np.sqrt(44 * std ** 2), size) ** 2 \
                + sps.norm.rvs(0, np.sqrt(44 * std ** 2), size) ** 2)
            
            # строим доверительный интервал
            left, right = solution(p, sample)

            # проверяем наличие ошибки, т. е. принадлежит ли истинное std доверительному интервалу
            error_counter += not [left < std < right][0]

            # измеряем также длину доверительного интервала
            ci_length.append(right - left)

        # смотрим на долю ошибок
        error_counter /= n

        # смотрим на среднее длин доверительных интервалов
        ci_length = np.array(ci_length).mean()

        # добавляем размер, доверие, частоту ошибок и длину интервала
        df.append([size, p, round(error_counter, 3), round(ci_length, 3)])

    # создаем датафрейм из наших данных и выводим в виде таблички   
    df = pd.DataFrame(df, columns=['Размер выборки', 'Доверие', 'Частота ошибок', 'Длина интервала'])

    return df

In [6]:
experiment(5000, 1)

,Размер выборки,Доверие,Частота ошибок,Длина интервала
0,1000,0.99,0.011,0.082
1,1000,0.90,0.102,0.052
2,100,0.70,0.295,0.104
3,100,0.90,0.096,0.165
4,10,0.95,0.048,0.672
5,10,0.90,0.102,0.553


# Как проводить А-Б тесты: Проверка гипотез. Задача 1

Очевидно, здесь нужно выбрать только корректный метод. Так как исторических данных нет, то просто приведем правильный код:

In [7]:
def solution(x_success: int, 
             x_cnt: int, 
             y_success: int, 
             y_cnt: int) -> bool:
    
    # тест для сравнений конверсий с одной классической сложной левосторонней альтернативой 
    # (конверсия на тесте меньше, чем на контроле)

    _, pval = proportions_ztest([x_success, y_success], [x_cnt, y_cnt], alternative='larger')

    # выявляем наличие эффекта
    effect = (pval < 0.1)
    
    return effect

# Как проводить А-Б тесты: Проверка гипотез. Задача 2

Считаем данные:

In [8]:
# считываем данные
hystorical_data = pd.read_csv('historical_data.csv', index_col=0)
modified_data_1 = pd.read_csv('modified_data_of_type_1.csv', index_col=0)
modified_data_2 = pd.read_csv('modified_data_of_type_2.csv', index_col=0)
modified_data_3 = pd.read_csv('modified_data_of_type_3.csv', index_col=0)

# проверяем ошибку несовпадения размерностей данных (если ошибка есть - выводим ее)
assert(len(hystorical_data) == len(modified_data_1) == len(modified_data_2) == len(modified_data_3))

# если ошибки нет, то все корректно и все данные имеют одинаковый размер
n = len(hystorical_data)

In [9]:
# создадим датафрейм, куда будем складывать результаты тестов
data = pd.DataFrame()

data['X'] = ['Историческое', 
             'Историческое', 
             'Историческое', 
             'Историческое']

data['Y'] = ['Историческое', 
             'Измененное типа 1', 
             'Измененное типа 2', 
             'Измененное типа 3']

data['Выборка'] = [300, 
                   300, 
                   300, 
                   300]

data.head()

,X,Y,Выборка
0,Историческое,Историческое,300
1,Историческое,Измененное типа 1,300
2,Историческое,Измененное типа 2,300
3,Историческое,Измененное типа 3,300


In [10]:
# сделаем список тестов, которые будем использовать

test_list = [{
    "p_value": lambda x, y: sps.ks_2samp(x, y, alternative="two-sided").pvalue,
    "name": "Kolmogorov-Smirnov"
}, {
    "p_value": lambda x, y: sps.anderson_ksamp([x, y]).pvalue,
    "name": "Anderson"
}, {
    "p_value": lambda x, y: sps.cramervonmises_2samp(x, y).pvalue,
    "name": "CVM"
}, {
    "p_value": lambda x, y: Energy(compute_distance="euclidean").test(x, y)[1],
    "name": "Energy with euclidean metric"
}, {
    "p_value": lambda x, y: Energy(compute_distance="minkowski", p=0.3).test(x, y)[1],
    "name": "Energy with minkowski metric"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="rbf", gamma=1/10).test(x, y)[1],
    "name": "MMD, rbf, gamma=0.1"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="rbf", gamma=1).test(x, y)[1],
    "name": "MMD, rbf, gamma=1"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="rbf", gamma=10).test(x, y)[1],
    "name": "MMD, rbf, gamma=10"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="laplacian", gamma=1/10).test(x, y)[1],
    "name": "MMD, laplacian, gamma=0.1"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="laplacian", gamma=1).test(x, y)[1],
    "name": "MMD, laplacian, gamma=1"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="laplacian", gamma=10).test(x, y)[1],
    "name": "MMD, laplacian, gamma=10"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="poly").test(x, y)[1],
    "name": "MMD, poly"
}]

In [11]:
even = np.arange(0, 300, 2)
odd = np.arange(1, 300, 2)

for test in test_list:
    counter_0 = 0
    counter_1 = 0
    counter_2 = 0
    counter_3 = 0

    for i in range(n):
        # считываем по одной строчке из исторических данных и из трех измененных данных

        x = np.array(hystorical_data.iloc[i])
        y_1 = np.array(modified_data_1.iloc[i])
        y_2 = np.array(modified_data_2.iloc[i])
        y_3 = np.array(modified_data_3.iloc[i])

        # чтобы сделать сравнение исторические и исторические, возьмем две выборки с четными
        # и нечетными индексами у исторических данных

        x_1 = x[even]
        x_2 = x[odd]

        # проводим четыре теста и учитываем эффект, если он есть

        counter_0 += test['p_value'](x_1, x_2) < 0.05
        counter_1 += test['p_value'](x, y_1) < 0.05
        counter_2 += test['p_value'](x, y_2) < 0.05
        counter_3 += test['p_value'](x, y_3) < 0.05

    # добавляем столбец с названием метода и соответствующими ошибками 
    # если сравниваем исторические против исторических, то эффекта быть не должно

    data[test['name']] = [counter_0 / n, (n - counter_1) / n, (n - counter_2) / n, (n - counter_3) / n]

In [12]:
data

,X,Y,Выборка,Kolmogorov-Smirnov,Anderson,CVM,Energy with euclidean metric,Energy with minkowski metric,"MMD, rbf, gamma=0.1","MMD, rbf, gamma=1","MMD, rbf, gamma=10","MMD, laplacian, gamma=0.1","MMD, laplacian, gamma=1","MMD, laplacian, gamma=10","MMD, poly"
0,Историческое,Историческое,300,0.035,0.055,0.055,0.050,0.050,0.045,0.05,0.050,0.05,0.050,0.050,0.05
1,Историческое,Измененное типа 1,300,0.055,0.015,0.025,0.020,0.020,0.015,0.02,0.025,0.02,0.020,0.090,0.02
2,Историческое,Измененное типа 2,300,0.165,0.090,0.105,0.110,0.110,0.110,0.10,0.125,0.11,0.115,0.205,0.07
3,Историческое,Измененное типа 3,300,0.260,0.145,0.220,0.225,0.225,0.360,0.32,0.205,0.22,0.205,0.260,0.22


Отсюда уже делаем выводы, что критерий Андерсона работает лучше всего в данном кейсе.

# Как проводить А-Б тесты: Проверка гипотез. Задача 3

Считаем данные:

In [13]:
# считаем данные
sample = np.array(pd.read_csv('hyp3_historical_data.csv').iloc[0, 1:]).astype('float')

# размер выборки
default_sample_size = len(sample)

Построим графики, где будем проверять конечность матожидания и дисперсии:

In [14]:
sample_stat = [{
    "function": lambda x: x.mean(),
    "description": "Среднее"
}, {
    "function": lambda x: (x**2).mean(),
    "description": "Среднее квадратов"
}]

min_sample_size = 1
max_sample_size = default_sample_size - 1

for stat in sample_stat:
    stat_value = stat["function"](sample)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[min_sample_size, max_sample_size-1], 
                             y=[stat_value, stat_value], 
                             name="Вся выборка"))

    stat_est_list = []
    for i in range(min_sample_size, max_sample_size):
        stat_est_list.append(stat["function"](sample[:i]))

    fig.add_trace(go.Scatter(x=np.arange(min_sample_size, max_sample_size), 
                             y=stat_est_list, 
                             name="Подвыборка"))

    fig.update_xaxes(type="log")
    fig.update_layout(title=stat["description"],
                      xaxis_title="Размер выборки",
                      yaxis_title="Значение оценки")
    fig.show()

Несложно увидеть, что второй момент ведет себя плохо и, скорее всего, он бесконечный, а первый момент ведет себя лучше и есть гипотеза, что как раз он и конечный. Поэтому попробуем взять `permutation_test` (в лекциях он себя показывал довольно неплохо и в среднем был лучше других):

In [15]:
def solution(x: np.array, y: np.array) -> bool:
    # смотрим на эффект
    effect = sps.permutation_test((x, y), lambda x, y, axis: np.mean(x, axis=axis) - np.mean(y, axis=axis), 
                 vectorized=True, 
                 n_resamples=1000,
                 alternative='greater').pvalue < 0.07
    
    return effect